In [10]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
import time
import builtins

# ==============================================================================
# STEP 1: CONFIGURATION
# ==============================================================================
ROWS_TO_GENERATE = 1000000 
TARGET_TABLE = "Lake24.dbo.person"

print(f"🎬 Starting generation of {ROWS_TO_GENERATE:,} records...")

# ==============================================================================
# STEP 2: LIVE METADATA ANALYSIS (DYNAMIC)
# ==============================================================================
# Load the target table once to get both schema and current stats
target_table_df = spark.table(TARGET_TABLE)
actual_schema = target_table_df.schema

# Get the LIVE count and MAX ID
stats_df = target_table_df.select(
    F.count("*").alias("current_count"), 
    F.max("person_id").alias("max_id")
).collect()[0]

initial_row_count = stats_df["current_count"]
max_id_val = stats_df["max_id"]
start_id = (max_id_val or 0) + 1

print(f"🔍 Table Analysis:")
print(f"   - Existing Records: {initial_row_count:,}")
print(f"   - Current Max ID:   {max_id_val:,}")
print(f"   - Resuming from:    {start_id:,}")

# ==============================================================================
# STEP 3: VECTORIZED SYNTHETIC GENERATION
# ==============================================================================
print(f"⚡ Generating {ROWS_TO_GENERATE:,} rows in-memory...")
start_time = time.time()

df_synth = spark.range(0, ROWS_TO_GENERATE) \
    .withColumn("person_id", (F.lit(start_id) + F.col("id")).cast(IntegerType())) \
    .withColumn("gender_rand", F.rand()) \
    .withColumn("gender_concept_id", F.when(F.col("gender_rand") > 0.52, 8532).otherwise(8507).cast(IntegerType())) \
    .withColumn("year_of_birth", (F.rand() * 70 + 1950).cast(IntegerType())) \
    .withColumn("month_of_birth", (F.rand() * 11 + 1).cast(IntegerType())) \
    .withColumn("day_of_birth", (F.rand() * 27 + 1).cast(IntegerType())) \
    .withColumn("race_rand", F.rand()) \
    .withColumn("race_concept_id", 
        F.when(F.col("race_rand") < 0.60, 8527)
         .when(F.col("race_rand") < 0.74, 8515)
         .otherwise(0).cast(IntegerType())) \
    .withColumn("ethnicity_concept_id", F.lit(38003564).cast(IntegerType())) \
    .withColumn("person_source_value", F.concat(F.lit("SYNTH_"), F.col("person_id").cast("string"))) \
    .withColumn("gender_source_value", F.when(F.col("gender_concept_id") == 8507, "M").otherwise("F")) \
    .withColumn("gender_source_concept_id", F.col("gender_concept_id").cast(IntegerType())) \
    .withColumn("birth_datetime", F.to_date(
        F.concat_ws("-", F.col("year_of_birth"), F.col("month_of_birth"), F.col("day_of_birth"))
    ))

# Align columns based on the actual schema profile
final_columns = []
for field in actual_schema.fields:
    if field.name in df_synth.columns:
        final_columns.append(F.col(field.name).cast(field.dataType))
    else:
        final_columns.append(F.lit(None).cast(field.dataType).alias(field.name))

final_df = df_synth.select(final_columns)

# ==============================================================================
# STEP 4: APPEND TO DELTA TABLE
# ==============================================================================
try:
    final_df.write.format("delta").mode("append").saveAsTable(TARGET_TABLE)
    execution_time = builtins.round(time.time() - start_time, 2)
    print(f"✅ SUCCESS: Processed in {execution_time} seconds.")
except Exception as e:
    print(f"❌ Error: {str(e)}")

# ==============================================================================
# STEP 5: FINAL REPORT (DYNAMIC SUMMARY)
# ==============================================================================
final_count = spark.table(TARGET_TABLE).count()

print("\n" + "="*40)
print("📊 SYNTHETIC DATA GENERATION SUMMARY")
print("="*40)
print(f"Target Table:     {TARGET_TABLE}")
print(f"Initial Rows:     {initial_row_count:,}")
print(f"Added Rows:       {ROWS_TO_GENERATE:,}")
print(f"Final Row Count:  {final_count:,}")
print(f"Execution Speed:  {builtins.round(ROWS_TO_GENERATE/execution_time, 0):,} rows/sec")
print("="*40)

StatementMeta(, 35da6aa5-e6a2-4bd8-9766-a4c53fe575f1, 12, Finished, Available, Finished, False)

🎬 Starting generation of 1,000,000 records...
🔍 Table Analysis:
   - Existing Records: 14,712,818
   - Current Max ID:   311,960,608
   - Resuming from:    311,960,609
⚡ Generating 1,000,000 rows in-memory...
✅ SUCCESS: Processed in 2.58 seconds.

📊 SYNTHETIC DATA GENERATION SUMMARY
Target Table:     Lake24.dbo.person
Initial Rows:     14,712,818
Added Rows:       1,000,000
Final Row Count:  15,712,818
Execution Speed:  387,597.0 rows/sec
